- 랭체인을 활용하여 해당 그래프 DB에서 데이터를 검색하고 활용해보겠습니다.
- 로컬검색과 글로벌검색을 직접 구현


In [2]:
from langchain_neo4j import Neo4jGraph
from langchain.chains import GraphCypherQAChain
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores.neo4j_vector import Neo4jVector
from dotenv import load_dotenv
import os

load_dotenv()

NEO4J_URI = os.getenv("NEO4J_URI")
NEO4J_AUTH = (os.getenv("NEO4J_USERNAME"), os.getenv("NEO4J_PASSWORD"))
NEO4J_DATABASE = "neo4j"

In [3]:
embedding = OpenAIEmbeddings(model="text-embedding-3-small")

# Neo4j 그래프 객체 생성
graph = Neo4jVector.from_existing_graph(
    embedding=embedding,
    node_label="__Entity__",
    text_node_properties=["description"],
    embedding_node_property="embedding",
    url=NEO4J_URI,
    username=NEO4J_AUTH[0],
    password=NEO4J_AUTH[1],
)

# Neo4j Graph 겍체 추가 생성
neo4j_graph = Neo4jGraph(
    url=NEO4J_URI,
    username=NEO4J_AUTH[0],
    password=NEO4J_AUTH[1],
    database=NEO4J_DATABASE,
)

- 먼저 로컬 검색을 구현해보겠습니다.
- 로컬 검색은 사용자의 질문과 가장 관련성이 높은 엔티티들을 중심으로 답변을 생성하는 방식입니다.
- 따라서 임베딩된 질문과 가장 유사한 엔티티들을 찾은 후, 해당 엔티티와 연관된 다양한 정보를 수집해야 합니다.
- 첫 단계로 엔티티와 연관된 다양한 정보를 수집하는 함수를 정의합니다.
- 해당 엔티티와 연결된 텍스트 청크, 커뮤니티 보고서 그리고 관련된 다른 엔티티 정보를 조회합니다.


In [5]:
def fetch_entity_context(entity_name):
    context = {"name": entity_name}
    try:
        #텍스트 청크 가져오기
        chunk_query = """
        MATCH (e: __Entity__ {name: $entity_name})<-[:HAS_ENTITY]-(c:__Chunk__)
        RETURN c.text AS text
        """
        chunk_result = neo4j_graph.query(chunk_query, {"entity_name": entity_name})
        context["text_chunks"] = [r["text"] for r in chunk_result] if chunk_result else ["No text chunk available"]

        # 커뮤니티 보고서 가져오기
        community_query = """
        MATCH (e: __Entity__ {name: $entity_name})-[:IN_COMMUNITY]->(com: __Community__)
        RETURN com.full_content AS report
        """
        community_result = neo4j_graph.query(community_query, {"entity_name": entity_name})
        context["community_reports"] = [r["report"] for r in community_result] if community_result else ["No community report available"]

        # 관련 엔티티 가져오기
        related_query = """
        MATCH (e: __Entity__ {name: $entity_name})-[:RELATED]-> (related: __Entity__)
        RETURN related.name AS name, related.description AS description
        """
        related_result = neo4j_graph.query(related_query, {"entity_name": entity_name})
        context["related_entities"] = (
            [{"name": r["name"], "description": r["description"]} for r in related_result]
            if related_result else []
        )
    except Exception as e:
        context["error"] = f"Error fetching context: {str(e)}"
    return context


- 다음으로 수집된 정보를 읽기 쉬운 구조로 정리하는 create_structured_context 함수를 정의하여, 여러 엔티티의 정보를 하나의 통합된 텍스트로 변환할 수 있도록합니다.


In [6]:
def create_structured_context(all_contexts, query):
    context_str = "## 질문과 관련된 엔티티 정보\n\n"
    context_str += "아래는 질문에 답변하는 데 유용한 엔티티들의 구조화된 정보입니다:\n\n"

    for i, ctx in enumerate(all_contexts, 1):
        context_str += f"### 엔티티 {i}: {ctx['name']}\n"
        context_str += f"- **설명**: {ctx['description']}\n"
        context_str += f"- **텍스트 청크**:\n"
        for chunk in ctx["text_chunks"]:
            context_str += f"  - {chunk}\n"
        context_str += f"- **커뮤니티 보고서**:\n"
        for report in ctx["community_reports"]:
            context_str += f"  - {report}\n"
        if ctx["related_entities"]:
            context_str += "- **관련 엔티티**:\n"
            for rel in ctx["related_entities"]:
                context_str += f"  - {rel['name']}: {rel['description']}\n"
        else:
            context_str += "- **관련 엔티티**: 없음\n"
        context_str += "\n"
    return context_str


- 이제 사용할 대규모 언어 모델을 설정하고, 생성해놓은 Neo4j 그래프 객체를 리트리버로 지정하여 정보 검색에 활용합니다.


In [7]:
# LLM 설정
llm = ChatOpenAI(model="gpt-4o-mini")

#리트리버 설정
retriever = graph.as_retriever(search_typye="similarity", search_kwargs={"k": 3})

- 전체 질의응답 흐름을 구현합니다.
- 벡터 스토어를 활용해 리트리버를 구성하여 질문과 유사한 엔티티를 검색합니다.
- 검색된 엔티티에 대해 fetch_entity_context 함수를 호출해 텍스트 청크, 커뮤니티 보고서, 관련 엔티티 정보를 수집
- 이를 기반으로 create_structured_context함수를 통해 구조화된 컨텍스트를 생성합니다.


In [11]:
# 질문 설정
query = "마일당 순이익(NET INCOME PER MILE)을 어떻게 분석해야 하나요?"
results = retriever.get_relevant_documents(query)

all_contexts = []
for result in results:
    entity_name = result.metadata.get("name", "Unknown")
    description = result.page_content
    context = fetch_entity_context(entity_name)
    context["name"] = entity_name
    context["description"] = description
    all_contexts.append(context)

# 구조화된 컨텍스트 생성
context_str = create_structured_context(all_contexts, query)

# LLM 프롬프트 작성
prompt = f"아래 맥락에 기반해서, 주어진 질문에 한국어로 답하세요\n\n**질문**: {query}\n\n**맥락**\n{context_str}"

# LLM 호출
response = llm.invoke(prompt)
print("Final Response:")
print(response.content)

Final Response:
마일당 순이익(NET INCOME PER MILE) 분석은 투자자에게 여러 중요한 요소를 고려하는 과정입니다. 이 지표는 특정 투자에서 발생하는 순이익을 마일 단위로 나눈 값으로, 투자 수익성을 평가하는 데 유용합니다. 아래는 마일당 순이익을 효과적으로 분석하기 위해 고려해야 할 몇 가지 주요 포인트입니다:

1. **수익성 평가**: 마일당 순이익은 각 마일에 대해 얼마나 많은 순이익을 얻고 있는지를 나타내므로, 이는 투자 수익성을 직접적으로 평가하는 데 유용합니다. 투자자들은 이 값을 통해 특정 투자의 효율성을 비교할 수 있습니다.

2. **비용 분석**: 순이익을 계산할 때 발생하는 비용을 면밀히 분석해야 합니다. 여기에는 운영비용, 유지보수비용, 기타 관련 비용이 포함됩니다. 비용을 명확히 이해함으로써 마일당 순이익을 보다 정확하게 산출할 수 있습니다.

3. **투자 전략과의 연계**: 마일당 순이익을 분석할 때, 투자자의 다른 투자 전략과의 연계성을 고려해야 합니다. 이를 통해 특정 투자가 전체 포트폴리오에 어떻게 영향을 미칠지를 평가할 수 있습니다.

4. **시장 동향**: 마일당 순이익에는 시장의 변화나 외부 경제 요인도 영향을 미칠 수 있으므로, 이러한 시장 동향을 따라가는 것이 중요합니다.

5. **위험 관리**: 높은 마일당 순이익이 항상 안정성이나 안전성을 보장하는 것은 아닙니다. 따라서, 투자자는 위험 요소를 충분히 이해하고 관리하여 최적의 결정을 내려야 합니다.

이러한 요소들을 종합적으로 고려하면 마일당 순이익 분석을 통해 더 나은 투자 결정을 내릴 수 있으며, 궁극적으로 재정적인 목표를 달성하는 데 큰 도움이 될 것입니다.


- 글로벌 검색을 구현해보겠습니다.


In [12]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-4o-mini")

- 글로벌 검색 구현에는 데이터셋 전체를 아우르는 질문에 대응하기 위해 맵-리듀스 방식을 채택합니다.
- 미리 정의된 MAP_SYSTEM_PROMPT와 map_prompt를 활용해 각 커뮤니티 리포트에서 중간 응답들을 생성합니다.
- 이 과정은 맵 단계에 해당합니다.


In [13]:
MAP_SYSTEM_PROMPT = """
---역할---
제공된 컨텍스트를 참고하여 사용자의 질문에 답하는 어시스턴트입니다.

---목표---
주어진 컨텍스트가 질문에 답하기에 적절하다면 질문에 대한 답을 한 뒤, 답변의 중요도 점수를 기입하여 JSON 형식으로 생성하세요.
정보가 부족하면 "모르겠습니다"라고 답하세요.
각 포인트는 다음을 포함해야 합니다:
- 답변: 질문에 대한 답변
- 중요도 점수: 0~100 사이의 정수
데이터 참조 예:
"예시 문장 [Data: Reports (2, 7, 64, 46, 34, +more)]"
(한 참조에 5개 이상의 id는 "+more"를 사용)
출력 예:
{{"Answer": "답변 [Data: Reports (보고서 id들)]", "score": 점수}}
"""

map_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", MAP_SYSTEM_PROMPT),
        ("human", "question: {question}\n\ncontext: {context}"),
    ]
)
map_chain = map_prompt | llm | StrOutputParser()

- 이어서 REDUCE_SYSTEM_PROMPT와 reduce_prompt를 통해 맵 단계에서 생성된 여러 분석가의 보고서를 종합하여 최종응답을 생성하는 리듀스 단계를 구현합니다.
- 이 단계에서는 핵심 포인트들을 통합하여 마크다운 형식의 응답으로 재구성합니다.


In [14]:
REDUCE_SYSTEM_PROMPT = """
---역할---
맵 단계에서 처리된 여러 결과를 종합하여 사용자의 질문에 답하는 어시스턴트입니다.

---목표---
제공된 맵 단계 결과를 바탕으로, 질문에 대한 종합적인 답변을 마크다운 형식으로 작성하세요.
중요도 점수를 고려하여 핵심적인 결과 위주로 반영하며, 불필요한 세부 사항은 제외하세요.
핵심 포인트와 시사점을 포함하고, 정보가 부족한 경우 "모르겠습니다"라고 답하세요.

--맵 단계 결과--
{report_data}
데이터 참조 형식은 아래를 따르세요:
예시 문장 [Data: Reports (2, 7, 34, 46, 64, +more)]
(참조 ID가 5개 이상인 경우 "+more"를 사용)
대상 응답 길이 및 형식: {response_type}"""

reduce_prompt = ChatPromptTemplate.from_messages(
    [
        ("system", REDUCE_SYSTEM_PROMPT),
        ("human", "{question}"),
    ]
)

reduce_chain = reduce_prompt | llm | StrOutputParser()

- 글로벌 검색 함수를 구현합니다.
- Neo4jGraph를 통해 특정 레벨의 커뮤니티 리포트를 조회한 뒤, 각 리포트에 대해 맵 체인을 적용하고 그 결과를 리듀스 체인으로 통합하여 최종 답변을 생성합니다.


In [16]:
from tqdm import tqdm
response_type: str = "multiple paragraphs"

def global_retriever(query: str, level: int, response_type: str = response_type) -> str:
    community_data = graph.query(
        """
        MATCH (c: __Community__)
        WHERE c.level = $level
        RETURN c.full_content AS output
        """,
        params={"level": level},
    )
    intermediate_results = []
    for community in tqdm(community_data, desc="Processing communities"):
        intermediate_result = map_chain.invoke(
            {
                "question": query,
                "context": community["output"],
            }
        )
        intermediate_results.append(intermediate_result)

    final_reponse = reduce_chain.invoke(
        {
            "report_data": intermediate_results,
            "question": query,
            "response_type": response_type,
        }
    )
    return final_reponse

print(global_retriever("이 책 주제가 뭐야?", 1))


Processing communities: 100%|██████████| 37/37 [01:34<00:00,  2.56s/it]


제공된 데이터를 종합해보면, 책의 주제는 다음과 같이 다양합니다. 주요 주제들은 다음과 같습니다:

1. **철도 산업**: 미국의 철도 산업과 그것을 규제하는 간섭통상위원회(ICC)의 역할, 주요 철도 회사들의 재정 구조 및 상호 의존성, 철도의 금융 관행 등이 다뤄집니다. 이는 국가의 교통망에서 철도가 가진 중요성을 강조하고 있습니다. 
   - **관련 데이터**: 철도 산업과 관련된 다양한 재무 지표, 금융 상품, 재조직화 과정 등이 포함됩니다. [Data: Reports (1, 4, 8, 12, 19)]

2. **투자 전략**: 투자 원칙과 전략, 특히 투자자들이 복잡한 투자 환경을 극복하고 자산을 최적화하는 방법에 대해 논의합니다. 금융 상품 간의 연관성과 투자자 신뢰의 중요성도 강조되고 있습니다.
   - **관련 데이터**: 유가증권과 관련된 투자 전략 및 개인 투자자와 재무 자문가 간의 상호작용도 포함됩니다. [Data: Reports (2, 9, 11, 18)]

3. **경제 동향**: 경제 성장과 투자 기회, 일반 경제 조건과 투자 결정 간의 상호 작용에 대한 논의가 이루어집니다. 다양한 금융 요소들의 관계와 경제적 요인이 시장 성과에 미치는 영향을 분석합니다.
   - **관련 데이터**: 경제 조건과 투자 결정의 상호 작용이 포함되며, 투자 관리가 기업 재무 건강에 미치는 영향도 다룹니다. [Data: Reports (3, 5, 14)]

4. **전자 문학 및 민주화**: Project Gutenberg와 관련하여 전자 문학의 자유 배급과 민주화를 주제로 하는 내용도 포함됩니다. [Data: Reports (20, 22, 23)]

이처럼 이 책은 철도 산업, 투자 전략, 경제 동향, 전자 문학과 같은 다양한 주제를 포괄하고 있으며, 각 주제는 서로 연결되어 있습니다.
